# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Mentoría 16 - ¿Cómo identificar fuga de ventas? Inteligencia artificial aplicada al sector comercial.

### Explorando Patrones de Datos a través de Clustering (TP3)

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---
## Librerías

In [ ]:
# Para que las funciones se actualicen sin tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

# Funciones de visualización y curación
import pandas as pd
import json
from os.path import exists
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
from scipy.stats import skew, kurtosis, skewtest, kurtosistest

# Funciones de clustering
from sklearn.cluster import KMeans, MeanShift
from sklearn import manifold, preprocessing, decomposition

# Funciones propias
from utils_limpieza import * 

# Clear preferencias
plt.rcdefaults()
pd.reset_option('^display\.float_format')

---
# Preparación de Datos <span style="color:magenta">**(Paso 1)**</span>

Se repiten los pasos de curación realizados en la entrega anterior, con algunas modificaciones que se creen necesarias.

1. Cargamos el dataset crudo

In [ ]:
path = '../data/raw/tp2_muestra_diplodatos_ventas_omega_modelo_2023.csv'
ventas = pd.read_csv(path)

2. Eliminamos las variables `INSCRIPCION`, `CATEGORIA`, `DESCRIPCION_CATEGORIA`, `CATEGORIA (Ajustado)`, `NOMBRE`, `'CM04`, `DESC_TRATAMIENTO_FISCAL`, `TRATAMIENTO_DIFERNCIAL`, `TRATAMIENTO_FISCAL` y `PORCENTAJE_COMISION_EMPRESA`.

In [ ]:
ventas_renamed = limpiar_basic(ventas, cols_drop=['INSCRIPCION', 'CATEGORIA', 
                                                  'DESCRIPCION_CATEGORIA', 
                                                  'CATEGORIA (Ajustado)', 
                                                  'NOMBRE', 'CM04', 
                                                  'DESC_TRATAMIENTO_FISCAL', 
                                                  'TRATAMIENTO_DIFERNCIAL', 
                                                  'TRATAMIENTO_FISCAL', 
                                                  'PORCENTAJE_COMISION_EMPRESA'])

3. Renombramos como `Otros` las subcategorías que no tengan al menos 1 `MODELO` = 1.

In [ ]:
ventas_renamed = renombrar_elementos(ventas_renamed, 
                                     columna='SUB-CATEGORIA', 
                                     fill_otros='Otros')

4. Eliminamos los registros que contienen `Otros` o `Instalación, Mantenimiento, Reparación, etc de productos varios` (a.k.a. `Mantenimiento`) como subcategoría.

In [ ]:
ventas_renamed = ventas_renamed[ventas_renamed['SUB-CATEGORIA'] != 'Otros'].copy()
ventas_renamed = ventas_renamed[ventas_renamed['SUB-CATEGORIA'] != 'Instalación, Mantenimiento, Reparación, etc de productos varios'].copy()

5. Creamos la variable `Fecha`, que surge como:
    $$Fecha = Año + Mes$$

In [ ]:
ventas_renamed["Fecha"] = pd.to_datetime(ventas_renamed['MES'].astype(str) + '-' + ventas_renamed['AÑO'].astype(str), format='%m-%Y')
ventas_renamed = limpiar_basic(ventas_renamed, cols_drop=['MES', 'AÑO'])

6. Anonimizamos la variable sensible `ID_VENDEDOR`.

In [ ]:
ventas_hash, _ = anonimizar(ventas_renamed, 'ID_VENDEDOR')

7. Simplificamos el nombre de las variables.

In [ ]:
# Renombramos
with open("../references/tp2_column_dict.json") as column_dict_json:
    column_dict = json.load(column_dict_json)

ventas_hash.rename(columns = column_dict, inplace = True)

8. Simplificamos los valores en `Deposito`


In [ ]:
ventas_hash, _ = anonimizar(ventas_hash, 'Deposito')

9. Simplificamos las categorías en `Subrubro`.

In [ ]:
with open("../references/tp2_subrubro_dict.json") as subrubro_dict_json:
    subrubro_dict = json.load(subrubro_dict_json)

ventas_hash['Subrubro'] = ventas_hash['Subrubro'].replace(subrubro_dict)

10. acá va el paso nuevo

11. Eliminamos el efecto de la inflación. El procedimiento es el siguiente:
    $$\text{VAR}_{mm-aaaa} \frac{\text{IPC}_{06-2022}}{\text{IPC}_{mm-aaaa}}$$

Para todo valor de cada variable (VAR) correspondiente a cierto mes "mm-aaaa" se lo divide por el Índice de Precios al Consumidor (IPC) correspondiente a ese mes "mm-aaaa" y luego se lo multiplica por el IPC correspondiente al mes "06-2022".

De esta forma, todos los valores de cada variable VAR van a quedar expresados en unidades monetarias del "06-2022".



In [ ]:
ventas_ipc = ventas_hash.copy()
# ventas_ipc["Fecha"] = pd.to_datetime(ventas_ipc["Fecha"])

In [ ]:
precios_path = "../data/external/tp2_IPC_Indec.csv"

if exists(precios_path):
    print('Este archivo ya existe.')
else:
    print('Este archivo no existe: ¡Vamos a crearlo!')
    url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_06_23.xls"
    df = pd.read_excel(url, sheet_name="Índices IPC Cobertura Nacional", header=None, usecols="B:CA", skiprows=[0, 1, 2, 3, 4, 6, 7, 8], nrows=2)
    df = df.transpose()
    df.columns = ["Fecha", "INDICE"]
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    df.to_csv(precios_path, index=False)

In [ ]:
# Abrir dataset de precios
precios = pd.read_csv(precios_path)
precios.head()

In [ ]:
# Hay que asegurar que la variable clave tenga el mismo tipo en los dos dataframes
precios["Fecha"] = pd.to_datetime(precios["Fecha"])

In [ ]:
ventas_ipc = ventas_ipc.merge(precios[["Fecha", "INDICE"]], on="Fecha", how="left")

In [ ]:
indexar(ventas_ipc, 'Ventas')
indexar(ventas_ipc, 'Comision')

In [ ]:
# Se descartan las variables que no se usan
ventas_ipc = limpiar_basic(ventas_ipc, cols_drop=['Ventas', 'Comision', 'INDICE'])

# Se renombran las variables
ventas_ipc.rename(columns = {'Ventas_Real': 'Ventas', 
                              'Comision_Real': 'Comision'}, inplace = True)

# Se reacomodan las columnas
ventas_ipc = ventas_ipc[['ID', 'Omega', 'Subrubro', 'Fecha', 
                          'Deposito', 'Ventas', 'Comision', 'Modelo']]

ventas_ipc = ventas_ipc.sort_values(['Fecha', 'Ventas']).reset_index(drop=True)

In [ ]:
ventas_fisc = ventas_ipc.copy()
ventas_sub11 = ventas_fisc.copy()

12. Para cada combinación posible de `Subrubro`, `ID` y `Fecha`, vamos a:
* Sumar todos los valores de `Ventas`.
* Sumar todos los valores de `Comision`.
* Mantener los valores de `Modelo` y de `Omega`.

Al hacer esto, se agregan en una única fila todas las observaciones que pertenezcan a un mismo vendedor en una dada fecha bajo un cierto subrubro, más allá del depósito desde el que se realiza la venta.

In [ ]:
agregado = ventas_sub11.groupby(['Subrubro', 'ID', 'Fecha']).agg({
    'Omega': 'min',
    'Ventas': 'sum',
    'Comision': 'sum',
    'Modelo': 'min'
}).reset_index()

13. Existe la posibilidad de que algunos vendedores tengan siempre ventas nulas dentro de un mismo subrubro. Como no aportan información, nos deshacemos de estos registros. En efecto había 34650 registros que cumplían esta condición.

In [ ]:
agregado_no_nulo = agregado.groupby(['Subrubro', 'ID']).filter(lambda x: (x['Ventas'] != 0).any()).copy()

14. Deberíamos tener 42 observaciones por par ID/Subrubro, pero se ve al comienzo del dataframe anterior que falta, por ejemplo, el 5 del 2020 al vendedor 5 en "Com Varios". Esto pasa en varios casos: a veces hay meses faltantes. Vamos a imputar esos meses con valor 0 en ventas y comisión (esto es particularmente importante si luego vamos a hacer diferencias, por ej. para que no se encuentre con un vacío (genera diferencia vacía) o que me opere contra un mes que en realidad no es el anterior).

In [ ]:
promedio_observaciones = agregado_no_nulo.groupby(['Subrubro', 'ID']).size().mean()

# Mostrar el promedio de observaciones por combinación de ID y Subrubro
print(promedio_observaciones)

Antes de proceder a rellenar los casos vacíos con valores cero, vamos a crear una variable que nos permita en el fututo distinguir los valores originales de los imputados (`Dato_original`).

In [ ]:
agregado_no_nulo['Dato_original'] = 1

In [ ]:
# Contar las observaciones por combinación de ID y Subrubro
conteo_combinaciones = agregado_no_nulo.groupby(['Subrubro', 'ID']).size().reset_index(name='Conteo')

# Filtrar las combinaciones con menos de 42 observaciones
combinaciones_faltantes = conteo_combinaciones[conteo_combinaciones['Conteo'] < 42]

# Lista para almacenar las observaciones faltantes
observaciones_faltantes = []

# Iterar sobre las combinaciones faltantes
for _, combinacion in combinaciones_faltantes.iterrows():
    id_val = combinacion['ID']
    subrubro_val = combinacion['Subrubro']
    
    # Obtener fechas existentes y fechas faltantes
    fechas_existentes = agregado_no_nulo[(agregado_no_nulo['ID'] == id_val) & (agregado_no_nulo['Subrubro'] == subrubro_val)]['Fecha']
    fechas_faltantes = set(agregado_no_nulo['Fecha'].unique()) - set(fechas_existentes)
    
    # Agregar observaciones faltantes al DataFrame
    for fecha_faltante in fechas_faltantes:
        observacion = {
            'ID': id_val,
            'Subrubro': subrubro_val,
            'Fecha': fecha_faltante,
            'Ventas': 0,
            'Comision': 0,
        }
        observaciones_faltantes.append(observacion)

# Crear DataFrame con las observaciones faltantes
df_observaciones_faltantes = pd.DataFrame(observaciones_faltantes)

# Agregar observaciones faltantes al DataFrame agregado_no_nulo
agregado_limpio = pd.concat([agregado_no_nulo, df_observaciones_faltantes], ignore_index=True)

# Mostrar el nuevo DataFrame con las observaciones faltantes agregadas
agregado_limpio

In [ ]:
# Imputo los valores Omega y Modelo asociados a los nuevos meses adicionados. Es más eficiente agregarlo en una celda aparte y no en el for (antes demoraba 9 minutos, ahora menos de 1).
agregado_limpio['Omega'] = agregado_limpio.groupby(['ID', 'Subrubro'])['Omega'].transform('max')
agregado_limpio['Modelo'] = agregado_limpio.groupby(['ID', 'Subrubro'])['Modelo'].transform('max')

# Relleno los valores nulos de Dato_original con 0.
agregado_limpio['Dato_original'] = agregado_limpio['Dato_original'].fillna(0)

agregado_limpio

In [ ]:
# Convertir las columnas 'Omega', 'Modelo' y 'Dato_original' a tipo int64
agregado_limpio['Omega'] = agregado_limpio['Omega'].astype('int64')
agregado_limpio['Modelo'] = agregado_limpio['Modelo'].astype('int64')
agregado_limpio['Dato_original'] = agregado_limpio['Dato_original'].astype('int64')

In [ ]:
promedio_observaciones = agregado_limpio.groupby(['Subrubro', 'ID']).size().mean()

# Mostrar el promedio de observaciones por combinación de ID y Subrubro
print(promedio_observaciones)

15. Ordenar luego de todo lo que se hizo. Dejar que primero esté Fecha. Sirve para el pivoteo de la próxima sección.

In [ ]:
registros_vendedores_abs = agregado_limpio.sort_values(['Fecha', 'Subrubro', 'ID']).reset_index(drop=True)
registros_vendedores_abs

16. Queremos hacer el cálculo de la varación porcentual. Con el dataset como está actualmente (presencia de ceros), pueden generarse resultados que tienden a infinito. Para superar esta situación:
* Los "ceros al comienzo" (el vendedor aún no formaba parte de la plataforma) se reemplazaron por NaN.
* Los "ceros en el medio" (vendedor imputa venta nula) se reemplazaron por -1. Al principio íbamos a poner +1, pero existe un registro que tenía este valor.

In [ ]:
registros_vendedores_rel = registros_vendedores_abs.copy()

In [ ]:
# Demora poquito más de 2 min

idx_borrar = np.array([])
for rubro in registros_vendedores_rel['Subrubro'].unique():
    print(rubro)
    ids = registros_vendedores_rel[registros_vendedores_rel['Subrubro'] == rubro]['ID'].unique()

    for id in ids:
        a = registros_vendedores_rel[(registros_vendedores_rel['Subrubro'] == rubro) & (registros_vendedores_rel['ID'] == id)]
        afec = a[a['Ventas'] > 0]['Fecha'].iloc[0]
        aidx = a.index[a['Ventas'] > 0][0]
        bidx = a.index[a['Ventas'] == 0]
        cidx = bidx[bidx > aidx]
        idx_borrar = np.concatenate([idx_borrar, cidx])

idx_borrar = np.array(idx_borrar)
idx_borrar = np.sort(idx_borrar)

In [ ]:
registros_vendedores_rel[['Ventas', 'Comision']] = registros_vendedores_rel[['Ventas', 'Comision']].replace({0:np.NaN})
# La siguiente línea es para cambiar los ceros del medio por -1
# registros_vendedores_rel.loc[idx_borrar, :] = registros_vendedores_rel.loc[idx_borrar, :].replace({np.NaN:-1})
registros_vendedores_rel

17. Cálculo de cambios porcentuales. La función pct_change devuelve en el rango normal, no en el porcentual

In [ ]:
crear_diferencia_porcentual(registros_vendedores_rel, 'Ventas', 12)
crear_diferencia_porcentual(registros_vendedores_rel, 'Comision', 12)
crear_diferencia_porcentual(registros_vendedores_rel, 'Ventas', 4)
crear_diferencia_porcentual(registros_vendedores_rel, 'Comision', 4)
registros_vendedores_rel

18. Pivotear los datos

In [ ]:
pivotear = registros_vendedores_rel.copy()

# Extraer el mes y el año de la columna "Fecha"
pivotear["Fecha"] = pd.to_datetime(pivotear["Fecha"])
pivotear["Month"] = pivotear["Fecha"].dt.month
pivotear["Year"] = pivotear["Fecha"].dt.year

# Convertir "Month" a string con formato de dos cifras
pivotear["Month"] = pivotear["Month"].apply(lambda x: str(x).zfill(2))

# Convertir "Year" a string y quedarse con los últimos 2 dígitos
pivotear["Year"] = pivotear["Year"].apply(lambda x: str(x)[-2:])

# Crear la variable "Fecha2" que concatena "Year" y "Month"
pivotear["Fecha2"] = pivotear["Year"] + pivotear["Month"]

# Eliminar columnas
pivotear.drop(columns=['Fecha','Ventas','Comision','Dato_original','Month','Year'], inplace=True)

# Renombrar la columnas
pivotear.rename(columns={'Fecha2': 'Fecha', 'Y_pct_Ventas': 'Y_pct_Ven', 'Y_pct_Comision': 'Y_pct_Com', 'F_pct_Ventas': 'F_pct_Ven', 'F_pct_Comision': 'F_pct_Com'}, inplace=True)

pivotear

In [ ]:
# Realizar el pivoteo y la agrupación
pivot_df = pivotear.pivot_table(index=["ID",'Subrubro','Omega','Modelo'], columns=['Fecha'], values=['Y_pct_Ven', 'Y_pct_Com', 'F_pct_Ven', 'F_pct_Com'])

# Generar los nombres de las columnas finales
columns = [f"{col[0]}_{col[1]}" for col in pivot_df.columns]

# Asignar los nuevos nombres de columnas
pivot_df.columns = columns

# Restablecer el índice para que "ID" vuelva a ser una columna
pivot_df = pivot_df.reset_index()

pivot_df

No se hace el cálculo de promedio y varianza de los vectores.

---

Se sintetizan todos los pasos de curación, a fin de tener a mano en caso de tener que revisar alguno de ellos. (Por hacer)

In [ ]:
vectores = pd.read_csv('../data/interim/tp2_vendedores_vector_resumen.csv')
vectores.head()

In [ ]:
vectores.shape

In [ ]:
# Definir grupos de variables:

F_Com = ['F_pct_Com_1905', 'F_pct_Com_1906', 'F_pct_Com_1907', 'F_pct_Com_1908', 
         'F_pct_Com_1909', 'F_pct_Com_1910', 'F_pct_Com_1911', 'F_pct_Com_1912',
         'F_pct_Com_2001', 'F_pct_Com_2002', 'F_pct_Com_2003', 'F_pct_Com_2004',
         'F_pct_Com_2005', 'F_pct_Com_2006', 'F_pct_Com_2007', 'F_pct_Com_2008',
         'F_pct_Com_2009', 'F_pct_Com_2010', 'F_pct_Com_2011', 'F_pct_Com_2012',
         'F_pct_Com_2101', 'F_pct_Com_2102', 'F_pct_Com_2103', 'F_pct_Com_2104',
         'F_pct_Com_2105', 'F_pct_Com_2106', 'F_pct_Com_2107', 'F_pct_Com_2108',
         'F_pct_Com_2109', 'F_pct_Com_2110', 'F_pct_Com_2111', 'F_pct_Com_2112',
         'F_pct_Com_2201', 'F_pct_Com_2202', 'F_pct_Com_2203', 'F_pct_Com_2204',
         'F_pct_Com_2205', 'F_pct_Com_2206']

F_Ven = ['F_pct_Ven_1905', 'F_pct_Ven_1906', 'F_pct_Ven_1907', 'F_pct_Ven_1908', 
         'F_pct_Ven_1909', 'F_pct_Ven_1910', 'F_pct_Ven_1911', 'F_pct_Ven_1912',
         'F_pct_Ven_2001', 'F_pct_Ven_2002', 'F_pct_Ven_2003', 'F_pct_Ven_2004',
         'F_pct_Ven_2005', 'F_pct_Ven_2006', 'F_pct_Ven_2007', 'F_pct_Ven_2008',
         'F_pct_Ven_2009', 'F_pct_Ven_2010', 'F_pct_Ven_2011', 'F_pct_Ven_2012',
         'F_pct_Ven_2101', 'F_pct_Ven_2102', 'F_pct_Ven_2103', 'F_pct_Ven_2104',
         'F_pct_Ven_2105', 'F_pct_Ven_2106', 'F_pct_Ven_2107', 'F_pct_Ven_2108',
         'F_pct_Ven_2109', 'F_pct_Ven_2110', 'F_pct_Ven_2111', 'F_pct_Ven_2112',
         'F_pct_Ven_2201', 'F_pct_Ven_2202', 'F_pct_Ven_2203', 'F_pct_Ven_2204',
         'F_pct_Ven_2205', 'F_pct_Ven_2206']

Y_Com = ['Y_pct_Com_2001', 'Y_pct_Com_2002', 'Y_pct_Com_2003', 'Y_pct_Com_2004',
         'Y_pct_Com_2005', 'Y_pct_Com_2006', 'Y_pct_Com_2007', 'Y_pct_Com_2008',
         'Y_pct_Com_2009', 'Y_pct_Com_2010', 'Y_pct_Com_2011', 'Y_pct_Com_2012',
         'Y_pct_Com_2101', 'Y_pct_Com_2102', 'Y_pct_Com_2103', 'Y_pct_Com_2104',
         'Y_pct_Com_2105', 'Y_pct_Com_2106', 'Y_pct_Com_2107', 'Y_pct_Com_2108',
         'Y_pct_Com_2109', 'Y_pct_Com_2110', 'Y_pct_Com_2111', 'Y_pct_Com_2112',
         'Y_pct_Com_2201', 'Y_pct_Com_2202', 'Y_pct_Com_2203', 'Y_pct_Com_2204',
         'Y_pct_Com_2205', 'Y_pct_Com_2206']

Y_Ven = ['Y_pct_Ven_2001', 'Y_pct_Ven_2002', 'Y_pct_Ven_2003', 'Y_pct_Ven_2004',
         'Y_pct_Ven_2005', 'Y_pct_Ven_2006', 'Y_pct_Ven_2007', 'Y_pct_Ven_2008',
         'Y_pct_Ven_2009', 'Y_pct_Ven_2010', 'Y_pct_Ven_2011', 'Y_pct_Ven_2012',
         'Y_pct_Ven_2101', 'Y_pct_Ven_2102', 'Y_pct_Ven_2103', 'Y_pct_Ven_2104',
         'Y_pct_Ven_2105', 'Y_pct_Ven_2106', 'Y_pct_Ven_2107', 'Y_pct_Ven_2108',
         'Y_pct_Ven_2109', 'Y_pct_Ven_2110', 'Y_pct_Ven_2111', 'Y_pct_Ven_2112',
         'Y_pct_Ven_2201', 'Y_pct_Ven_2202', 'Y_pct_Ven_2203', 'Y_pct_Ven_2204',
         'Y_pct_Ven_2205', 'Y_pct_Ven_2206']

basics = ['Modelo', 'Subrubro']

In [ ]:
#vectores_interes = vectores[basics + Y_Ven]
vectores_interes = vectores[Y_Ven]
vectores_interes.head()

In [ ]:
vectores_interes.info()

Hay problemas con los vacíos, para que me los tome el modelo:

In [ ]:
# Ventas anuales
vectores_interes = vectores[Y_Ven]
msno.matrix(vectores_interes, fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
# Ventas anuales(zoom)
vectores_interes = vectores[Y_Ven]
msno.matrix(vectores_interes[185:190], fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
vectores.iloc[186, :]

In [ ]:
vectores['Y_pct_Ven_2109'].max()

In [ ]:
vectores[vectores['Y_pct_Ven_2109'] == 203697461.3233292]

In [ ]:
ventas_fisc = pd.read_csv('../data/interim/tp2_ventas_fisc.csv')
ventas_fisc[(ventas_fisc['ID'] == 186) & (ventas_fisc['Subrubro'] == 'Com. Varios')]

In [ ]:
ventas_fisc[(ventas_fisc['ID'] == 186)]['Subrubro'].value_counts()

**HACER**
- Llevar a cero todo lo que está entre -1000 y +1000 (esto antes del tratamiento de inflación)

20/9 >> 2.287884e-02
21/9 >> 4.660361e+06
3017	1	Gondola de dónde viene para dar ese 10-2?



In [ ]:
ventas_fisc[ventas_fisc['ID'] == 3017]

In [ ]:
# Ventas cuatrimestrales
vectores_interes = vectores[F_Ven]
msno.matrix(vectores_interes, fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
# Comisiones anuales
vectores_interes = vectores[Y_Com]
msno.matrix(vectores_interes, fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
# Comisiones cuatrimestrales
vectores_interes = vectores[F_Com]
msno.matrix(vectores_interes, fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
msno.bar(vectores_interes, fontsize=12, color="tab:blue", figsize=(10, 4))
plt.show()

In [ ]:
# Paso 1: Contar valores vacíos por fila
vacios_por_fila = vectores_interes.isnull().sum(axis=1)

# Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
resumen = vacios_por_fila.value_counts().reset_index()
resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')
resumen.head()

# ** Chequear **  >> Esto borra teniendo en cuenta yearly

In [ ]:
plt.figure(figsize=(10, 4))  # Tamaño del gráfico
plt.bar(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], color='blue', alpha=0.7)
plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Vacíos por Vendedor')
plt.xticks(resumen['Cantidad de Valores Vacíos'])
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Muestra el gráfico
plt.show()


También hay problema de outliers. Qué significa que una variación me dé -20.000%. Que tenía ej 1.000.000 y pasé a 0,05?. Algo hay que hacer

In [ ]:
vectores_interes.describe()

In [ ]:
# # Configuración del diseño del gráfico
# plt.figure(figsize=(16, 10))  # Tamaño del layout

# # Itera a través de cada columna y crea un KDE plot
# for column in vectores_interes.columns:
#     plt.subplot(5, 7, vectores_interes.columns.get_loc(column) + 1)  # 5 filas, 7 columnas
#     sns.kdeplot(vectores_interes[column], fill=True)
#     plt.title(column)

# # Ajusta la disposición y muestra el gráfico
# plt.tight_layout()
# plt.show()

In [ ]:
# Configuración del diseño del gráfico
plt.figure(figsize=(16, 10))  # Tamaño del layout

# Itera a través de cada columna y crea un boxplot
for i, column in enumerate(vectores_interes.columns):
    plt.subplot(5, 7, i + 1)  # 5 filas, 7 columnas
    sns.boxplot(x=vectores_interes[column])
    plt.title(column)

# Ajusta la disposición y muestra el gráfico
plt.tight_layout()
plt.show()

---
# Selección del Número de Clusters <span style="color:magenta">**(Paso 2)**</span>

### Elbow method

Para probar, le quito los vacíos.

In [ ]:
vectores_interes_sin_vacios.describe()

In [ ]:
gondola = vectores[vectores['Subrubro'] == 'Gondola']

#[basics + Y_Ven]
vectores_interes = gondola[Y_Ven]

In [ ]:
vectores_interes_sin_vacios = vectores_interes.dropna().copy()
print(vectores_interes_sin_vacios.shape)

# me quedo con el +-500% que se ve como un 5 acá OJO
vec_pm500 = vectores_interes_sin_vacios.where((vectores_interes_sin_vacios < 5) & (vectores_interes_sin_vacios > - 5))
vec_pm500 = vec_pm500.dropna()

vectores_interes_sin_vacios = vec_pm500

In [ ]:
vec_pm500.describe()

In [ ]:
#Prueba: para elegir el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i, n_init=10).fit(vectores_interes_sin_vacios).inertia_ for i in range(2,12)]

plt.plot(np.arange(2, 12), scores)
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("Inertia of k-Means versus number of clusters")

### Coeficiente Silhouette

---
## Aplicación de Modelos de Clustering <span style="color:magenta">**(Paso 3)**</span>

### K-means

In [ ]:
km = KMeans(n_clusters=3, n_init=10) # El parámetro n_init igual a 10 me lo pide para no tirar error. Ver luego qué implica.
km.fit(vectores_interes_sin_vacios)
clusters = km.labels_

In [ ]:
vectores_clusters = vectores_interes_sin_vacios.copy()
vectores_clusters['kmeans_4'] = km.labels_
print('Kmeans encontró: ', max(km.labels_)+1, 'clusters, nosotros forzamos la cantidad')
vectores_clusters.head(4)

In [ ]:
# Esto está hecho metiendo a todos los rubros en la misma bolsa
# Hacer por subrubro
vectores_clusters['kmeans_4'].value_counts()

In [ ]:
std_scale=preprocessing.StandardScaler().fit(vec_pm500)
X_scaled=std_scale.transform(vec_pm500) # numpyarray Estandarizado (le resta la media y divide por el desvío) por columna

pca=decomposition.PCA(n_components=4) #elegimos 2, 3 o 4 pero pueden ser más,

pca.fit(X_scaled) #input data is centered but not scaled for each feature before applying the SVD

# proporción de varianza
print('proporción de varianza por componente: ', pca.explained_variance_ratio_)
# proporción de varianza acumulada
print ('proporción de varianza por componente acumulada: ', pca.explained_variance_ratio_.cumsum())

X_projected=pca.transform(X_scaled) #numpy array
print ('tamaño de los datos: ', X_projected.shape)


In [ ]:
X_tsne_kmeans_4 = pd.DataFrame(X_projected)
X_tsne_kmeans_4['kmeans_4'] = km.labels_

sns.scatterplot(data=X_tsne_kmeans_4,
                x=0, 
                y=1, 
                hue="kmeans_4", 
                palette="deep")#, alpha=0.25)

# sns.pairplot(X_projected)

In [ ]:
from sklearn import (manifold, preprocessing, decomposition)

tsne = manifold.TSNE(n_components=2, verbose=1,perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(vec_pm500)

In [ ]:
X_tsne_kmeans_4 = pd.DataFrame(X_tsne)
X_tsne_kmeans_4['kmeans_4'] = km.labels_

sns.scatterplot(data=X_tsne_kmeans_4,
                x=0, 
                y=1, 
                hue="kmeans_4", 
                palette="deep")

---


In [ ]:
gondola = vectores[vectores['Subrubro'] == 'Gondola']

#[basics + Y_Ven]
vectores_interes = gondola[F_Ven]

In [ ]:
vectores_interes_sin_vacios = vectores_interes.dropna().copy()
print(vectores_interes_sin_vacios.shape)

# me quedo con el +-500% que se ve como un 5 acá OJO
vec_pm500 = vectores_interes_sin_vacios.where((vectores_interes_sin_vacios < 5) & (vectores_interes_sin_vacios > - 5))
vec_pm500 = vec_pm500.dropna()

vectores_interes_sin_vacios = vec_pm500

In [ ]:
vec_pm500.describe()

In [ ]:
#Prueba: para elegir el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i, n_init=10).fit(vectores_interes_sin_vacios).inertia_ for i in range(2,12)]

plt.plot(np.arange(2, 12), scores)
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("Inertia of k-Means versus number of clusters")

### Coeficiente Silhouette

---
## Aplicación de Modelos de Clustering <span style="color:magenta">**(Paso 3)**</span>

### K-means

In [ ]:
km = KMeans(n_clusters=3, n_init=10) # El parámetro n_init igual a 10 me lo pide para no tirar error. Ver luego qué implica.
km.fit(vectores_interes_sin_vacios)
clusters = km.labels_

In [ ]:
vectores_clusters = vectores_interes_sin_vacios.copy()
vectores_clusters['kmeans_4'] = km.labels_
print('Kmeans encontró: ', max(km.labels_)+1, 'clusters, nosotros forzamos la cantidad')
vectores_clusters.head(4)

In [ ]:
# Esto está hecho metiendo a todos los rubros en la misma bolsa
# Hacer por subrubro
vectores_clusters['kmeans_4'].value_counts()

In [ ]:
pca=decomposition.PCA(n_components=2) #elegimos 2, 3 o 4 pero pueden ser más,

pca.fit(vec_pm500) #input data is centered but not scaled for each feature before applying the SVD

# proporción de varianza
print('proporción de varianza por componente: ', pca.explained_variance_ratio_)
# proporción de varianza acumulada
print ('proporción de varianza por componente acumulada: ', pca.explained_variance_ratio_.cumsum())

X_projected=pca.transform(vec_pm500) #numpy array
print ('tamaño de los datos: ', X_projected.shape)


In [ ]:
X_tsne_kmeans_4 = pd.DataFrame(X_projected)
X_tsne_kmeans_4['kmeans_4'] = km.labels_

sns.scatterplot(data=X_tsne_kmeans_4,
                x=0, 
                y=1, 
                hue="kmeans_4", 
                palette="deep")#, alpha=0.25)

# sns.pairplot(X_projected)

In [ ]:
from sklearn import (manifold, preprocessing, decomposition)

tsne = manifold.TSNE(n_components=2, verbose=1,perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(vec_pm500)

In [ ]:
X_tsne_kmeans_4 = pd.DataFrame(X_tsne)
X_tsne_kmeans_4['kmeans_4'] = km.labels_

sns.scatterplot(data=X_tsne_kmeans_4,
                x=0, 
                y=1, 
                hue="kmeans_4", 
                palette="deep")

In [ ]:
selected_columns = [Y_Ven[0], Y_Ven[1], 'kmeans_4']
sns.pairplot(vectores_clusters[selected_columns], hue='kmeans_4')

Probando para un solo rubro..

In [ ]:
vectores_interes = vectores[basics + Y_Ven]
vectores_gondola = vectores_interes[vectores_interes['Subrubro'] == 'Gondola'].copy()
vectores_gondola = vectores_gondola.dropna().copy()
vectores_gondola

In [ ]:
vectores_gondola = vectores_gondola[Y_Ven].copy()

In [ ]:
#Prueba: para elegir el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i, n_init=10).fit(vectores_gondola).inertia_ for i in range(2,12)]

plt.plot(np.arange(2, 12), scores)
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("Inertia of k-Means versus number of clusters")

In [ ]:
km = KMeans(n_clusters=4, n_init=10) # El parámetro n_init igual a 10 me lo pide para no tirar error. Ver luego qué implica.
km.fit(vectores_gondola)
clusters = km.labels_

In [ ]:
vectores_clusters = vectores_gondola.copy()
vectores_clusters['kmeans_4'] = km.labels_
print('Kmeans encontró: ', max(km.labels_)+1, 'clusters, nosotros forzamos la cantidad')
vectores_clusters.head(4)

In [ ]:
vectores_clusters['kmeans_4'].value_counts()

In [ ]:
selected_columns = [Y_Ven[0], Y_Ven[1], 'kmeans_4']
sns.pairplot(vectores_clusters[selected_columns], hue='kmeans_4')

### Otros algoritmos de clustering

---
## Visualización de Resultados <span style="color:magenta">**(Paso 4)**</span>

---
## Interpretación y Evaluación <span style="color:magenta">**(Paso 5)**</span>

---
## Confianza en los Resultados <span style="color:magenta">**(Paso 6)**</span>

---
## Preguntas finales <span style="color:magenta">**(Paso 7)**</span>